<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/test_sample_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install wget
! pip install datasets

In [ ]:
! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

In [2]:
%cd PyTorch-Architectures/

/content/PyTorch-Architectures


In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from modeling_AlexNet.model import AlexNet
from toolkit.custom_dataset_cv import DataLoaderCIFAR10Classification
from toolkit.utils import dict_to_device
from toolkit.trainer import Trainer

In [4]:
# Hyperparameters
INPUT_SIZE = 128
BATCH_SIZE = 128
LR = 0.0001
EPOCHS = 10

In [5]:
train_loader = DataLoaderCIFAR10Classification(resize=INPUT_SIZE, train=True)
train_loader = train_loader.return_dataloader(batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoaderCIFAR10Classification(resize=INPUT_SIZE, train=False)
valid_loader = valid_loader.return_dataloader(batch_size=BATCH_SIZE, shuffle=False)

print(f"Length of Train Loader --> {len(train_loader)}")
print(f"Length of Valid Loader --> {len(valid_loader)}")

cifar10 extracted...
cifar10 exists...
Length of Train Loader --> 391
Length of Valid Loader --> 79


In [6]:
# Sanity check DataLoader
for sample in train_loader:
  assert sample["pixel_values"].dim() == 4
  assert sample["pixel_values"].size(2) == INPUT_SIZE
  break

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=10)
model.to(device)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters --> {params}")

Trainable Parameters --> 57044810


In [8]:
# Sanity check forward pass
model.eval()
with torch.set_grad_enabled(False):
  for sample in train_loader:
    loss, logits = model(**dict_to_device(sample, device=device))
    print(f"Loss --> {loss.item(): .2f}")
    print(f"Logits Shape --> {logits.shape}")
    break

Loss -->  2.30
Logits Shape --> torch.Size([128, 10])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [9]:
trainer = Trainer(model=model,
                  train_dataset=train_loader,
                  valid_dataset=valid_loader,
                  )
trainer.train(lr=LR, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              shuffle_train=True,
              scheduler=None,
              metric="cv_accuracy"
              )

  0%|          | 0/3910 [00:00<?, ?it/s]

Epoch: 1 || Training Loss: 1.642 || cv_accuracy: 51.090
Epoch: 2 || Training Loss: 1.173 || cv_accuracy: 61.890
Epoch: 3 || Training Loss: 0.945 || cv_accuracy: 69.530
Epoch: 4 || Training Loss: 0.782 || cv_accuracy: 72.420
Epoch: 5 || Training Loss: 0.678 || cv_accuracy: 75.120
Epoch: 6 || Training Loss: 0.569 || cv_accuracy: 76.210
Epoch: 7 || Training Loss: 0.497 || cv_accuracy: 77.970
Epoch: 8 || Training Loss: 0.419 || cv_accuracy: 79.220
Epoch: 9 || Training Loss: 0.361 || cv_accuracy: 78.520
Epoch: 10 || Training Loss: 0.292 || cv_accuracy: 79.610
